In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
import re

In [2]:
# Load the dataset
data = pd.read_csv('.\mbti_1.csv', encoding='latin1', error_bad_lines=False)  # Replace 'path_to_your_dataset.csv' with the actual path

# Display the first few rows of the dataset
print(data.head())

c:\Users\manya\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3326: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
b'Skipping line 163: expected 2 fields, saw 18\nSkipping line 244: expected 2 fields, saw 74\nSkipping line 491: expected 2 fields, saw 47\nSkipping line 873: expected 2 fields, saw 47\nSkipping line 890: expected 2 fields, saw 29\nSkipping line 1012: expected 2 fields, saw 52\nSkipping line 1234: expected 2 fields, saw 16\nSkipping line 1237: expected 2 fields, saw 8\nSkipping line 1306: expected 2 fields, saw 49\nSkipping line 1352: expected 2 fields, saw 21\nSkipping line 1725: expected 2 fields, saw 56\nSkipping line 1859: expected 2 fields, saw 5\nSkipping line 2006: expected 2 fields, saw 32\nSkipping line 2043: expected 2 fields, saw 24\nSkipping line 2280: expected 2 fields, saw 28\nSkipping line 2292: expected 2 fields, saw 29\nSkipping lin

   type                                              posts
0  INFJ  'http://www.youtube.com/watch?v=qsXHcwe3krw|||...
1  ENTP  'I'm finding the lack of me in these posts ver...
2  INTP  'Good one  _____   https://www.youtube.com/wat...
3  INTJ  'Dear INTP,   I enjoyed our conversation the o...
4  ENTJ  'You're fired.|||That's another silly misconce...


In [3]:
# Splitting text posts into individual posts
data['posts'] = data['posts'].apply(lambda x: x.split('|||'))

# Display the updated dataset with split posts
print(data.head())

   type                                              posts
0  INFJ  ['http://www.youtube.com/watch?v=qsXHcwe3krw, ...
1  ENTP  ['I'm finding the lack of me in these posts ve...
2  INTP  ['Good one  _____   https://www.youtube.com/wa...
3  INTJ  ['Dear INTP,   I enjoyed our conversation the ...
4  ENTJ  ['You're fired., That's another silly misconce...


In [5]:
# Optional: Remove commas from text posts using regular expressions
#data['posts'] = data['posts'].apply(lambda x: re.sub(r',', '', x))

tokenization + padding

In [7]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [8]:
# Splitting data into train and test sets
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Tokenization and Padding
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_data['posts'])

max_len = 100  # Adjust max length as needed
train_sequences = tokenizer.texts_to_sequences(train_data['posts'])
test_sequences = tokenizer.texts_to_sequences(test_data['posts'])

train_sequences_padded = pad_sequences(train_sequences, maxlen=max_len, padding='post', truncating='post')
test_sequences_padded = pad_sequences(test_sequences, maxlen=max_len, padding='post', truncating='post')

# Display some preprocessed data
print("Tokenized and Padded Sequences (Train):")
print(train_sequences_padded[:5])
print("Shape of Train Sequences Padded:", train_sequences_padded.shape)

Tokenized and Padded Sequences (Train):
[[1534 1535  443 1536 1537  444 1538 1539 1540 1541 1542 1543 1544 1545
  1546 1547 1548 1549 1550 1551 1552 1553 1554 1555 1556 1557 1558 1559
  1560 1561 1562 1563 1564 1565 1566 1567 1568 1569 1570 1571 1572 1573
  1574 1575 1576 1577 1578 1579 1580 1581    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0]
 [1582 1583 1584 1585 1586 1587 1588 1589 1590 1591  445 1592 1593 1594
  1595 1596 1597 1598 1599 1600 1601 1602 1603 1604 1605 1606 1607 1608
  1609 1610 1611 1612 1613 1614 1615 1616 1617 1618 1619 1620 1621 1622
  1623 1624 1625 1626 1627 1628 1629 1630    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0

encoding

In [9]:
# Encode MBTI type labels
label_encoder = LabelEncoder()
label_encoder.fit(data['type'])

train_labels = label_encoder.transform(train_data['type'])
test_labels = label_encoder.transform(test_data['type'])

# Display encoded labels
print("Encoded Train Labels:", train_labels[:5])

Encoded Train Labels: [ 9  0  9 10 11]


model definition

In [10]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout

# Define LSTM model
model = Sequential()
model.add(Embedding(len(tokenizer.word_index) + 1, 100, input_length=max_len))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(16, activation='softmax'))  # 16 output classes (MBTI types)

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [11]:
model.fit(train_sequences_padded, train_labels, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
173/173 [==============================] - 112s 621ms/step - loss: 2.3402 - accuracy: 0.1911 - val_loss: 2.2530 - val_accuracy: 0.2232
Epoch 2/10
173/173 [==============================] - 116s 672ms/step - loss: 2.2989 - accuracy: 0.1978 - val_loss: 2.2509 - val_accuracy: 0.2232
Epoch 3/10
173/173 [==============================] - 118s 681ms/step - loss: 2.2971 - accuracy: 0.1989 - val_loss: 2.2474 - val_accuracy: 0.2232
Epoch 4/10
173/173 [==============================] - 117s 677ms/step - loss: 2.2972 - accuracy: 0.1951 - val_loss: 2.2479 - val_accuracy: 0.2232
Epoch 5/10
173/173 [==============================] - 119s 687ms/step - loss: 2.2967 - accuracy: 0.1962 - val_loss: 2.2490 - val_accuracy: 0.2232
Epoch 6/10
173/173 [==============================] - 117s 674ms/step - loss: 2.2938 - accuracy: 0.1986 - val_loss: 2.2510 - val_accuracy: 0.2232
Epoch 7/10
173/173 [==============================] - 117s 678ms/step - loss: 2.2940 - accuracy: 0.2031 - val_loss: 2.2449 -

Evaluating model

In [13]:
test_loss, test_accuracy = model.evaluate(test_sequences_padded, test_labels)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

54/54 [==============================] - 2s 29ms/step - loss: 2.2778 - accuracy: 0.2278
Test Loss: 2.2777645587921143
Test Accuracy: 0.2278260886669159


In [14]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

In [15]:
# Predict probabilities for each class
y_pred_probabilities = model.predict(test_sequences_padded)

# Convert probabilities to class labels (argmax to get the index of the highest probability)
y_pred = np.argmax(y_pred_probabilities, axis=1)

# Calculate accuracy
accuracy = accuracy_score(test_labels, y_pred)
print(f"Accuracy: {accuracy:.4f}")

# Calculate precision, recall, and F1-score
precision = precision_score(test_labels, y_pred, average='weighted')
recall = recall_score(test_labels, y_pred, average='weighted')
f1 = f1_score(test_labels, y_pred, average='weighted')

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")

# Calculate confusion matrix
conf_matrix = confusion_matrix(test_labels, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

54/54 [==============================] - 2s 29ms/step
Accuracy: 0.2278
Precision: 0.0519
Recall: 0.2278
F1-score: 0.0845
Confusion Matrix:
[[  0   0   0   0   0   0   0   0   0  33   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0 114   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0  36   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0 126   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   8   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0  10   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   6   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0  21   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0 262   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0 393   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0 258   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0 244   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0  39   0   0   0   0

c:\Users\manya\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Five fold cross validation

In [16]:
from sklearn.model_selection import KFold, StratifiedKFold

In [17]:
# Define number of folds
n_splits = 5

# Initialize StratifiedKFold for classification
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

In [ ]:
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Define number of folds
n_splits = 5

# Initialize StratifiedKFold for classification
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Initialize lists to store evaluation metrics across folds
accuracies = []
precisions = []
recalls = []
f1_scores = []
confusion_matrices = []

for train_idx, val_idx in skf.split(data['posts'], data['type']):
    # Split data into train and validation sets based on fold indices
    X_train, X_val = data['posts'][train_idx], data['posts'][val_idx]
    y_train, y_val = data['type'][train_idx], data['type'][val_idx]

    # Encode MBTI type labels to numeric labels
    label_encoder = LabelEncoder()
    y_train_encoded = label_encoder.fit_transform(y_train)
    y_val_encoded = label_encoder.transform(y_val)

    # Tokenization and Padding
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(X_train)
    max_len = 100  # Adjust max length as needed

    X_train_sequences = tokenizer.texts_to_sequences(X_train)
    X_val_sequences = tokenizer.texts_to_sequences(X_val)

    X_train_padded = pad_sequences(X_train_sequences, maxlen=max_len, padding='post', truncating='post')
    X_val_padded = pad_sequences(X_val_sequences, maxlen=max_len, padding='post', truncating='post')

    # Define and compile LSTM model
    model = Sequential()
    model.add(Embedding(len(tokenizer.word_index) + 1, 100, input_length=max_len))
    model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(len(label_encoder.classes_), activation='softmax'))  # Output layer with number of classes

    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    # Train LSTM model
    model.fit(X_train_padded, y_train_encoded, epochs=10, batch_size=32, verbose=0)

    # Evaluate LSTM model on validation data
    y_pred_encoded = np.argmax(model.predict(X_val_padded), axis=1)

    # Decode predicted labels back to MBTI types
    y_pred_decoded = label_encoder.inverse_transform(y_pred_encoded)

    # Calculate evaluation metrics for this fold
    accuracy = accuracy_score(y_val, y_pred_decoded)
    precision = precision_score(y_val, y_pred_decoded, average='weighted')
    recall = recall_score(y_val, y_pred_decoded, average='weighted')
    f1 = f1_score(y_val, y_pred_decoded, average='weighted')
    conf_matrix = confusion_matrix(y_val, y_pred_decoded)

    # Append metrics to lists
    accuracies.append(accuracy)
    precisions.append(precision)
    recalls.append(recall)
    f1_scores.append(f1)
    confusion_matrices.append(conf_matrix)

    # Print or display metrics for this fold
    print(f"Fold Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}")
    print("Confusion Matrix:")
    print(conf_matrix)

# Calculate average metrics across all folds
avg_accuracy = np.mean(accuracies)
avg_precision = np.mean(precisions)
avg_recall = np.mean(recalls)
avg_f1 = np.mean(f1_scores)

# Print or display average metrics across all folds
print("\nAverage Metrics Across Folds:")
print(f"Average Accuracy: {avg_accuracy:.4f}")
print(f"Average Precision: {avg_precision:.4f}")
print(f"Average Recall: {avg_recall:.4f}")
print(f"Average F1-score: {avg_f1:.4f}")

# Display average confusion matrix
avg_conf_matrix = np.mean(confusion_matrices, axis=0)
print("Average Confusion Matrix:")
print(avg_conf_matrix)